In [12]:
from keras.models import load_model
import tensorflow as tf
from fastapi import FastAPI, Body
from fastapi.middleware.cors import CORSMiddleware

from pydantic import BaseModel

from json import loads

from keras.models import load_model

import tensorflow as tf
import numpy as np
import spacy

from Objects.Analysers.Functions.get_word_pairs import get_wp_in_line_hard
import pandas as pd
from string import punctuation
import re
import string
from scipy.spatial.distance import cosine

In [2]:
nlp_type_of_event_extraction = spacy.load('../../PipeLines/WordsExtraction/words_extract/')
nlp_classic = spacy.load('ru_core_news_sm')
kcm_extraction_model = tf.keras.models.load_model(
    '../../PipeLines/Coder/coder_from_spacy_to_kcm_onh5.h5')
# kcm_extraction_model.evaluate
is_description_model = load_model('../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize.h5')

In [13]:
def sendStudentJson(description):
    def get_vectorized_wp_and_wp(text, vectorizer):
        list_of_wp = list()
        list_of_vectors = list()
        for sentence in text.split('.'):
            for wp in get_wp_in_line_hard(sentence):
                list_of_vectors.append(vectorizer(wp))
                list_of_wp.append(wp)
        return np.array(list_of_wp), np.array(list_of_vectors)
    wp_of_event, wp_of_event_vectorized = get_vectorized_wp_and_wp(description, lambda text: nlp_classic(text).vector)
    #is_descriptor
    is_wp_descriptor = is_description_model.predict(wp_of_event_vectorized) > 0.992
    #descriotors_vectorized
    descriptors_vectorized = wp_of_event_vectorized[is_wp_descriptor.T[0]]
    #predict
    predict = kcm_extraction_model.predict(descriptors_vectorized)
    #descriptors_name
    descriptors_name = wp_of_event[is_wp_descriptor.T[0]]

    #desc_without_punct
    def desc_without_punct(descriptors_name):
        desc = list()
        for i in descriptors_name:
            desc.append(i.translate(str.maketrans('','', string.punctuation)))
        return desc
    
    description_name_without_punct = desc_without_punct(descriptors_name)
    def vectorized_description_name_without_punct(description_name_without_punct):
        disc = list()
        for i in description_name_without_punct:
            disc.append(nlp_classic(i).vector)
        return disc
    vectorized_description_name_without_punctuation = vectorized_description_name_without_punct(description_name_without_punct)


    #distance
    n = 0
    final_summ = [list() for i in vectorized_description_name_without_punctuation]

    for i in range(len(vectorized_description_name_without_punctuation)):
        for j in range(i + 1, len(vectorized_description_name_without_punctuation)):
            cos = cosine(vectorized_description_name_without_punctuation[i], vectorized_description_name_without_punctuation[j])
            final_summ[i].append(cos)
            final_summ[j].append(cos)

    def stdotkl(final_summ):
        final_list = list()
        for i in final_summ:
            final_list.append(np.std(i))
        return final_list
    
    vectorize_end = stdotkl(final_summ)
    res = np.where(vectorize_end == np.min(vectorize_end))[0]
    description_name_without_punct[int(res)]
    
    return description_name_without_punct[int(res)], kcm_extraction_model.predict(np.array([nlp_classic(description_name_without_punct[int(res)]).vector]))

In [9]:
abc = 'В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным задачам, сравнительные характеристики.'

In [14]:
sendStudentJson(abc)

1/1 [==============================] - 0s 17ms/step


('представлений типов мышления',
 array([[ 0.3524445 ,  0.82012445, -0.16171165]], dtype=float32))